Handling different columns requiring different types of encoding is quite difficult. This is because when we do encoding it creates a numpy array, then we have merge them all which is quite cumbersome and inefficient

In [17]:
import numpy as np
import pandas as pd

In [18]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder

In [19]:
df=pd.read_csv('covid_toy.csv')

In [20]:
df.sample(5)

,age,gender,fever,cough,city,has_covid
42,27,Male,100.0,Mild,Delhi,Yes
67,65,Male,99.0,Mild,Bangalore,No
48,66,Male,99.0,Strong,Bangalore,No
35,82,Female,102.0,Strong,Bangalore,No
33,26,Female,98.0,Mild,Kolkata,No


In [21]:
df['cough'].value_counts()

cough
Mild      62
Strong    38
Name: count, dtype: int64

In [22]:
df['city'].value_counts()

city
Kolkata      32
Bangalore    30
Delhi        22
Mumbai       16
Name: count, dtype: int64

In [23]:
# cough=ordinal, city, gender=nominal

In [24]:
df.isnull().sum()

age           0
gender        0
fever        10
cough         0
city          0
has_covid     0
dtype: int64

In [25]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test=train_test_split(df.drop(columns=['has_covid']), df['has_covid'])

In [26]:
X_train

,age,gender,fever,cough,city
73,34,Male,98.0,Strong,Kolkata
67,65,Male,99.0,Mild,Bangalore
84,69,Female,98.0,Strong,Mumbai
94,79,Male,NaN,Strong,Kolkata
1,27,Male,100.0,Mild,Delhi
...,...,...,...,...,...
82,24,Male,98.0,Mild,Kolkata
47,18,Female,104.0,Mild,Bangalore
9,64,Female,101.0,Mild,Delhi
90,59,Female,99.0,Strong,Delhi


### Normal

In [27]:
# adding simple imputer to fever col
si = SimpleImputer()
X_train_fever = si.fit_transform(X_train[['fever']])

# also the test data
X_test_fever = si.fit_transform(X_test[['fever']])
                                 
X_train_fever.shape

(75, 1)

In [28]:
# Ordinalencoding -> cough
oe = OrdinalEncoder(categories=[['Mild','Strong']])
X_train_cough = oe.fit_transform(X_train[['cough']])

# also the test data
X_test_cough = oe.fit_transform(X_test[['cough']])

X_train_cough.shape

(75, 1)

In [31]:
# OneHotEncoding -> gender,city
ohe = OneHotEncoder(drop='first',sparse_output=False)
X_train_gender_city = ohe.fit_transform(X_train[['gender','city']])

# also the test data
X_test_gender_city = ohe.fit_transform(X_test[['gender','city']])

X_train_gender_city.shape

(75, 4)

In [32]:
# Extracting Age
X_train_age = X_train.drop(columns=['gender','fever','cough','city']).values

# also the test data
X_test_age = X_test.drop(columns=['gender','fever','cough','city']).values

X_train_age.shape

(75, 1)

In [33]:
X_train_transformed = np.concatenate((X_train_age,X_train_fever,X_train_gender_city,X_train_cough),axis=1)
# also the test data
X_test_transformed = np.concatenate((X_test_age,X_test_fever,X_test_gender_city,X_test_cough),axis=1)

X_train_transformed.shape

(75, 7)

### Column Transformer

In [34]:
from sklearn.compose import ColumnTransformer

In [39]:
transformer=ColumnTransformer(transformers=[
    ('tnf1',SimpleImputer(),['fever']),
    ('tnf2',OrdinalEncoder(categories=[['Mild','Strong']]),['cough']),
    ('tnf3',OneHotEncoder(sparse_output=False, drop='first'),['gender','city'])
], remainder='passthrough')

In [40]:
transformer.fit_transform(X_train)

array([[ 98.        ,   1.        ,   1.        ,   0.        ,
          1.        ,   0.        ,  34.        ],
       [ 99.        ,   0.        ,   1.        ,   0.        ,
          0.        ,   0.        ,  65.        ],
       [ 98.        ,   1.        ,   0.        ,   0.        ,
          0.        ,   1.        ,  69.        ],
       [100.89705882,   1.        ,   1.        ,   0.        ,
          1.        ,   0.        ,  79.        ],
       [100.        ,   0.        ,   1.        ,   1.        ,
          0.        ,   0.        ,  27.        ],
       [101.        ,   0.        ,   1.        ,   1.        ,
          0.        ,   0.        ,  15.        ],
       [103.        ,   1.        ,   1.        ,   0.        ,
          0.        ,   0.        ,  46.        ],
       [ 98.        ,   1.        ,   0.        ,   0.        ,
          1.        ,   0.        ,  71.        ],
       [100.89705882,   1.        ,   1.        ,   0.        ,
          1.    

In [42]:
transformer.transform(X_train).shape

(75, 7)

In [43]:
transformer.transform(X_test).shape

(25, 7)